In [1]:
library(sf)
library(ggplot2)
library(DBI)
library(odbc)
library(getPass)
library(RPostgreSQL)

Linking to GEOS 3.11.1, GDAL 3.6.1, PROJ 9.1.0; sf_use_s2() is TRUE

Warning message:
“package ‘odbc’ was built under R version 4.2.3”
Warning message:
“package ‘getPass’ was built under R version 4.2.3”


# Querying using R

Nivapy includes Python functions that make it easy to query data from relational databases connected to the Hub, including Nivabasen and PostGIS (see [here](https://nbviewer.org/github/NIVANorge/dstoolkit_cookbook/blob/master/notebooks/oracle_example.ipynb) and [here](https://nbviewer.org/github/NIVANorge/dstoolkit_cookbook/blob/master/notebooks/postgis_example.ipynb), respectively). Equivalent functions in R are not yet available, but you can still create database connections and run your own queries to achieve the same thing.

<div class="alert alert-success" role="alert">
  If you create your own functions in R, please consider sharing them on GitHub so that others can use them too!
</div>

# 1. Connecting to Nivabase



In [2]:
connect_oracle <- function() {
  #' Connect to Nivabase.
  #'
  #' Returns:
  #'     Database connection object  
  username = getPass("Username: ")
  password = getPass("Password: ")      
  conn <- dbConnect(
    odbc(),
    Driver = "/opt/conda/orahome/libsqora.so.12.1",
    DBQ = "DBORA-NIVA-PROD01.NIVA.CORP:1555/NIVABPRD",
    UID    = username,
    PWD   = password,
    Port   =  1555,
  )

  return(conn)
}

In [3]:
# Connect to Nivabasen
eng = connect_oracle()

Username:  ········
Password:  ········


In [4]:
result <- dbGetQuery(eng, "SELECT * from nivadatabase.projects")
head(result)

,PROJECT_ID,PROJECT_NAME,PROJECT_DESCRIPTION,STARTDATE,ENDDATE,ENTERED_BY,ENTERED_DATE
,<dbl>,<chr>,<chr>,<dttm>,<dttm>,<chr>,<dttm>
1,2380,Trondheimsfjorden 1540_83,1540_83,NA,NA,ABM,2007-06-14 13:03:34
2,2381,Trondheimsfjorden 1641_84,1641_84,NA,NA,ABM,2007-06-14 13:03:34
3,2382,Tvedestrand supp. sedimentund.,4986-2005,NA,NA,JVE,2012-05-08 13:26:00
4,2383,Varangerfjorden 2213_89,2213_89,NA,NA,ABM,2007-06-14 13:03:34
5,2384,Varangerfjorden 3281_95,3281_95,NA,NA,ABM,2007-06-14 13:03:34
6,2385,Vefsnfjorden 1330_81,1330_81,NA,NA,ABM,2007-06-14 13:03:34


## 2. Connecting to PostGIS

The JupyterHub has its own PostGIS database containing various useful geospatial datasets (see [here](https://github.com/NIVANorge/niva_jupyter_hub/blob/master/postgis_db/postgis_db_dataset_summary.md) for further details).

In [5]:
connect_postgis <- function(database = "general", user = "jovyan", password = "joyvan_ro_pw") {
  #' Connect to PostGIS as a "read-only" user. Should eventually provide similar functionality
  #' to nivapy.spatial.connect_postgis(), but this function is currently more limited.
  #'
  #' Args:
  #'     database: Str. Name of database to connect to
  #'
  #' Returns:
  #'     Database connection object
  conn <- dbConnect(
    PostgreSQL(),
    host = "postgis",
    port = 5432,
    dbname = database,
    user = user,
    password = password
  )

  return(conn)
}

In [6]:
# Connect to JupyterHub PostGIS database
pg_eng <- connect_postgis()

In [7]:
# Query projects for which catchments are available
proj_df <- st_read(pg_eng, c("niva", "projects"))
proj_df

Warning message in st_read.DBIObject(...):
“Could not find a simple features geometry column. Will return a `data.frame`.”


,project_id,project_code,project_name,aquamonitor_id,contact,description
,<int>,<chr>,<chr>,<int>,<chr>,<chr>
1,1,Refelv_Over,Overvåking av referanseelver,11226,NA,Overvåking av referanseelver
2,2,190246,CL Vestland,NA,KAU,Critical loads calculations for Vestland
3,4,O-190091,Nasjonal Innsjøundersøkelse 2019,12433,HIN,1000 Lakes survey in 2019
4,6,220263,Høyanger,NA,KAU,Critical loads calculations for Høyanger


In [8]:
# Get the stations associated with the 2019 '1000 Lakes survey' (project_id = 4)
sql <- ("SELECT * FROM niva.stations 
WHERE station_id IN ( 
SELECT station_id FROM niva.projects_stations 
WHERE project_id = 4 
)")
stns_gdf <- st_read(pg_eng, query = sql)
print(head(stns_gdf))

Simple feature collection with 6 features and 6 fields
Geometry type: POINT
Dimension:     XY
Bounding box:  xmin: 11.18603 ymin: 59.00512 xmax: 12.51199 ymax: 61.82058
Geodetic CRS:  WGS 84
  station_id station_code station_name aquamonitor_id longitude latitude
1        259      221-1-2    Langtjern          26070  11.85061 59.80992
2        260      101-2-7    Hokksjøen          26071  11.55995 59.00512
3        261     402-2-13   Sætertjern          26072  12.44367 60.05777
4        262     419-1-25    Mjøgsjøen          26073  11.84528 60.33203
5        263      425-2-2      Kottern          26074  12.51199 60.58732
6        264     432-1-26 Måsabutjørna          26075  11.18603 61.82058
                       geom
1 POINT (11.85061 59.80992)
2 POINT (11.55995 59.00512)
3 POINT (12.44367 60.05777)
4 POINT (11.84528 60.33203)
5 POINT (12.51199 60.58732)
6 POINT (11.18603 61.82058)


In [9]:
# Get catchment boundaries for stations in '1000 Lakes' project
cat_gdf <- st_read(pg_eng, c("niva", "catchments"))
cat_gdf <- cat_gdf[cat_gdf$station_id %in% stns_gdf$station_id, ]
print(head(cat_gdf))

Simple feature collection with 6 features and 1 field
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: 5.255444 ymin: 60.06819 xmax: 13.5029 ymax: 65.21352
Geodetic CRS:  WGS 84
   station_id                           geom
78        303 MULTIPOLYGON (((5.51852 60....
79        370 MULTIPOLYGON (((12.36239 60...
80        372 MULTIPOLYGON (((11.11143 61...
81        999 MULTIPOLYGON (((13.25816 65...
82        297 MULTIPOLYGON (((5.260024 60...
83        271 MULTIPOLYGON (((8.009185 61...


In [10]:
# Plot the first catchment in Leaflet
cat_gdf[1, ]

The legacy packages maptools, rgdal, and rgeos, underpinning this package
will retire shortly. Please refer to R-spatial evolution reports on
https://r-spatial.org/r/2023/05/15/evolution4.html for details.
This package is now running under evolution status 0 

Registered S3 method overwritten by 'geojsonsf':
  method        from   
  print.geojson geojson



,station_id,geom
,<int>,<MULTIPOLYGON [°]>
78,303,MULTIPOLYGON (((5.51852 60....
